
<h2>Coursera Capstone Project - The Battle of Neighbourhoods</h2>
This Project is about finding a suitable neighbourhood in the City of Toronto to open a Restaurant chain
<h1>Introduction</h1>
A group of 5 investors are plannning to open a restaurant chain in the city of Toronto. So they are interested to know which neighbourhood has most restaurants. And also to find the Neighbourhoods that are similar to the one which has most restaurants so as to be easy to decide upon the location.
<h3>Data</h3>
The location of each neighbourhood is scraped from wikipedia.The Borough name, neighbourhood name are taken from wikipedia page.
The latitude and longitude of each neighbourhood is ob Foursquare API is used to get the location of restaurants in the city of Toronto. And also to get the nearby top venues in each neighbourhood.
<h3>Methodology</h3>
1.Get the location of each Borough and Neighbourhood in Toronto using wikipedia page.
2.Find the number of Neighbourhoods in each Borough.
3.Get the restaurants locations and visualize it on map.
4.Select the Boroughs and Neighbourhoods that are nearby to the one with most restaurants by checking out the number of neghbourhoods and visualizing each neighbourhood on map.
5.Use Kmeans Clustering algorithm to cluster the neighbourhoods. And select the ones that are nearby to the popular area with most restaurants and the ones that are similar to it.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!pip install selenium

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


<h2>The data to be used is a json file uploaded unto my data in the cognitive lab.</h2>
We begin by reading the json data from the source and loading it into a pandas dataframe.

In [2]:
import pandas as pd
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df=pd.read_html(url, header=0)[0]

df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"



<h1>Clean the data</h1>


In [3]:
DF = df[df.Borough != 'Not assigned']
DF = df[df.Neighbourhood != 'Not assigned']
DF = DF[["Borough","Neighbourhood"]]
toronto_data = DF.reset_index(drop = True)
toronto_data.head()

,Borough,Neighbourhood
0,North York,Parkwoods
1,North York,Victoria Village
2,Downtown Toronto,"Regent Park, Harbourfront"
3,North York,"Lawrence Manor, Lawrence Heights"
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
Latitude = []
Longitude = []
for neigh,bou,i in zip(toronto_data['Neighbourhood'],toronto_data['Borough'],toronto_data.index):  
    address = "'"+neigh+ ","+ " "+ bou+"'"
    geolocator = Nominatim(user_agent="toronto_explorer")
    location = geolocator.geocode(address)
    if location != None:
        Latitude.append(location.latitude)
        Longitude.append(location.longitude)
        
    else:
        Latitude.append(0)
        Longitude.append(0)
        
print(Latitude)
print(Longitude)

[43.7611243, 43.732658, 0, 43.7165164, 0, 43.6642319, 43.8091955, 43.775347, 0, 0, 43.7087117, 0, 0, 43.775347, 43.69992, 0, 43.6883215, 0, 43.7534804, 43.6710244, 0, 0, 43.7598243, 43.7047983, 43.6617525, 0, 43.75646655, 43.7996637, 0, 43.704553, 0, 43.6602019, 43.7437422, 0, 0, 0, 0, 0, 0, 43.7691966, 43.7492988, 43.6602585, 0, 0, 0, 0, 43.7492988, 43.6691887, 0, 0, 43.7600778, 0, 0, 43.7492988, 43.6539542, 0, 0, 0, 0, 43.7739798, 43.7492988, 43.729199, 43.7001239, 43.6679579, 43.7001608, 0, 43.7440391, 43.697936, 43.701028, 43.6538668, 43.6936399, 0, 43.7739798, 0, 0, 43.6389385, 0, 0, 43.7853531, 43.697936, 0, 43.6517776, 0, 0, 0, 0, 0, 0, 0, 0, 0, 43.661357100000004, 0, 43.6017173, 43.72337025, 43.809343, 0, 0, 0, 43.661357100000004, 0, 0, 0]
[-79.3240594, -79.3111892, 0, -79.4422605, 0, -79.5319625, -79.2217008, -79.3459439, 0, 0, -79.4406853, 0, 0, -79.3459439, -79.319279, 0, -79.4280805, 0, -79.1799949, -79.296712, 0, 0, -79.2252908, -79.3680904, -79.3861073, 0, -79.22669244258

In [6]:
toronto_data['Latitude'] = Latitude
toronto_data['Longitude'] = Longitude
toronto_data = toronto_data[toronto_data.Latitude != 0].reset_index(drop = True)
toronto_data = toronto_data[toronto_data.Longitude != 0].reset_index(drop = True)
toronto_data

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.761124,-79.324059
1,North York,Victoria Village,43.732658,-79.311189
2,North York,"Lawrence Manor, Lawrence Heights",43.716516,-79.442261
3,Etobicoke,"Islington Avenue, Humber Valley Village",43.664232,-79.531963
4,Scarborough,"Malvern, Rouge",43.809196,-79.221701
5,North York,Don Mills,43.775347,-79.345944
6,North York,Glencairn,43.708712,-79.440685
7,North York,Don Mills,43.775347,-79.345944
8,East York,Woodbine Heights,43.699920,-79.319279
9,York,Humewood-Cedarvale,43.688322,-79.428080



<h1>Now that the data is cleaned and ready, will proceed further.</h1>
<h2>Lets get the Location of Toronto</h2>

In [9]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
Tlatitude = location.latitude
Tlongitude = location.longitude
print('Latitude =', Tlatitude)
print('Longitude =',Tlongitude)

Latitude = 43.6534817
Longitude = -79.3839347



<h2>Now get the Restaurants location and visualize it on the Map.</h2>
<h1>Using the Foursquare API to get the location</h1>

In [10]:
CLIENT_ID = 'MHUYK3THYV0W5VZXJOUBKELZMGS2OYCDOL5IKQBRGASOZP0F' # your Foursquare ID
CLIENT_SECRET = 'JYJVTEXPRGIONJVHLVW1SHMB1TYLJL5BRLRVC3CN2Y2L1U3R' # your Foursquare Secret
VERSION = '20190529'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MHUYK3THYV0W5VZXJOUBKELZMGS2OYCDOL5IKQBRGASOZP0F
CLIENT_SECRET:JYJVTEXPRGIONJVHLVW1SHMB1TYLJL5BRLRVC3CN2Y2L1U3R


In [11]:
search_query = 'restaurant'
radius = 50000
print(search_query + ' .... OK!')

restaurant .... OK!


In [12]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, Tlatitude, Tlongitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=MHUYK3THYV0W5VZXJOUBKELZMGS2OYCDOL5IKQBRGASOZP0F&client_secret=JYJVTEXPRGIONJVHLVW1SHMB1TYLJL5BRLRVC3CN2Y2L1U3R&ll=43.6534817,-79.3839347&v=20190529&query=restaurant&radius=50000&limit=30'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fbf7c669ec7747540fc6f9b'},
 'response': {'venues': [{'id': '4ad4c05ff964a52048f720e3',
    'name': 'Hemispheres Restaurant & Bistro',
    'location': {'address': '110 Chestnut Street',
     'lat': 43.65488413420439,
     'lng': -79.38593077371578,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65488413420439,
       'lng': -79.38593077371578}],
     'distance': 224,
     'postalCode': 'M5G 1R3',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['110 Chestnut Street',
      'Toronto ON M5G 1R3',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d14e941735',
      'name': 'American Restaurant',
      'pluralName': 'American Restaurants',
      'shortName': 'American',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1606384742',
    'hasPerk': False},
   {'id': '

In [14]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.neighborhood
0,4ad4c05ff964a52048f720e3,Hemispheres Restaurant & Bistro,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1606384742,False,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",224,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,NaN
1,5c33c7cd8194fc002c66f72e,Green Tea Restaurant Downtown,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1606384742,False,261 Spadina Avenue. Upper level,43.652488,-79.397501,"[{'label': 'display', 'lat': 43.65248783033554...",1098,M5T 2E3,CA,Toronto,ON,Canada,"[261 Spadina Avenue. Upper level, Toronto ON M...",NaN,NaN,NaN
2,5750b013498e755287c6de97,Some Time BBQ Grill Restaurant 碳烤屋,"[{'id': '52af3b773cf9994f4e043c03', 'name': 'S...",v-1606384742,False,988 Baldwin Street,43.655874,-79.393826,"[{'label': 'display', 'lat': 43.655874, 'lng':...",839,NaN,CA,Toronto,ON,Canada,"[988 Baldwin Street, Toronto ON, Canada]",NaN,NaN,NaN
3,4ad4c05cf964a52006f620e3,Victoria's Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1606384742,False,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",763,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,498556908,NaN
4,4ada5d5bf964a520e92121e3,The Hot House Restaurant & Bar,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1606384742,False,35 Church St,43.648824,-79.373702,"[{'label': 'display', 'lat': 43.64882370529773...",973,M5E 1T3,CA,Toronto,ON,Canada,"[35 Church St (at Front St E), Toronto ON M5E ...",at Front St E,NaN,NaN


In [15]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",224,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,4ad4c05ff964a52048f720e3
1,Green Tea Restaurant Downtown,Chinese Restaurant,261 Spadina Avenue. Upper level,43.652488,-79.397501,"[{'label': 'display', 'lat': 43.65248783033554...",1098,M5T 2E3,CA,Toronto,ON,Canada,"[261 Spadina Avenue. Upper level, Toronto ON M...",NaN,NaN,5c33c7cd8194fc002c66f72e
2,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,43.655874,-79.393826,"[{'label': 'display', 'lat': 43.655874, 'lng':...",839,NaN,CA,Toronto,ON,Canada,"[988 Baldwin Street, Toronto ON, Canada]",NaN,NaN,5750b013498e755287c6de97
3,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",763,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,NaN,4ad4c05cf964a52006f620e3
4,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,"[{'label': 'display', 'lat': 43.64882370529773...",973,M5E 1T3,CA,Toronto,ON,Canada,"[35 Church St (at Front St E), Toronto ON M5E ...",at Front St E,NaN,4ada5d5bf964a520e92121e3


In [16]:
dataframe_filtered.name

0                    Hemispheres Restaurant & Bistro
1                      Green Tea Restaurant Downtown
2                 Some Time BBQ Grill Restaurant 碳烤屋
3                              Victoria's Restaurant
4                     The Hot House Restaurant & Bar
5                                  Studio Restaurant
6                 Sky Dragon Chinese Restaurant 龍翔酒樓
7                 North-East Chinese Restaurant 華北美食
8                             Rol San Restaurant 龍笙棧
9                       Aroma Fine Indian Restaurant
10                       Cottage Restaurant & Lounge
11                            Azure Restaurant & Bar
12                           Victor Restaurant & Bar
13           Tasty's Caribbean Restaurant & Catering
14                           Swatow Restaurant 汕頭小食家
15                          Ka Chi Korean Restaurant
16                                    360 Restaurant
17                             Sightlines Restaurant
18                     Hong Shing Chinese Rest

In [20]:
!pip install folium
import pandas as pd
import folium
map_Neigh = folium.Map(location=[Tlatitude, Tlongitude], zoom_start=11)


for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='#000000',
        popup=label,
        fill = True,
        fill_color='#000000',
        fill_opacity=0.6
    ).add_to(map_Neigh)

map_Neigh

     |████████████████████████████████| 93 kB 3.0 MB/s  eta 0:00:01


AttributeError: module 'folium.features' has no attribute 'CircleMarker'


<h2>Now visualize the neighbourhoods and select the ones nearby.</h2>
<h1>First find the number of Neighbourhoods in each Borough</h1>

In [21]:
toronto_data.groupby(['Borough']).count()

,Neighbourhood,Latitude,Longitude
Borough,,,
Central Toronto,5,5,5
Downtown Toronto,3,3,3
East Toronto,4,4,4
East York,3,3,3
Etobicoke,4,4,4
North York,16,16,16
Scarborough,7,7,7
West Toronto,4,4,4
York,3,3,3


<h3>When we explore the above map, we find that the Borough - Central Toronto, Downtown Toronto, East Toronto, Etobicoke, York are the ones nearby to it. Hence we proceed only with those Boroughs.</h3>

In [22]:
CT_data = toronto_data[toronto_data['Borough'] == 'Central Toronto'].reset_index(drop=True)
CT_data

DT_data = toronto_data[toronto_data['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
DT_data

ET_data = toronto_data[toronto_data['Borough'] == 'East Toronto'].reset_index(drop=True)
ET_data


etobicoke_data = toronto_data[toronto_data['Borough'] == 'Etobicoke'].reset_index(drop=True)
etobicoke_data

Y_data = toronto_data[toronto_data['Borough'] == 'York'].reset_index(drop=True)
Y_data

,Borough,Neighbourhood,Latitude,Longitude
0,York,Humewood-Cedarvale,43.688322,-79.428080
1,York,"Runnymede, The Junction North",43.667958,-79.483114
2,York,Weston,43.700161,-79.516247


In [23]:
address = 'Central Toronto, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
CTlatitude = location.latitude
CTlongitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(CTlatitude, CTlongitude))

The geograpical coordinate of Central Toronto are 43.65238435, -79.38356765.


In [24]:
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
DTlatitude = location.latitude
DTlongitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(DTlatitude, DTlongitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [25]:
address = 'East Toronto, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
ETlatitude = location.latitude
ETlongitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(ETlatitude, ETlongitude))

The geograpical coordinate of Downtown Toronto are 43.626243, -79.396962.


In [26]:
address = 'Etobicoke, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
Elatitude = location.latitude
Elongitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(Elatitude, Elongitude))

The geograpical coordinate of Etobicoke are 43.6435559, -79.5656326.


In [27]:
address = 'York, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
Ylatitude = location.latitude
Ylongitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(Ylatitude, Ylongitude))

The geograpical coordinate of Etobicoke are 43.6896191, -79.479188.


In [28]:
# create map of Manhattan using latitude and longitude values
map_Neigh = folium.Map(location=[Tlatitude, Tlongitude], zoom_start=11)


folium.features.CircleMarker(
    [Elatitude, Elongitude],
    radius=10,
    color='red',
    popup='Etobicoke',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_Neigh)

# add markers to map
for lat, lng, label in zip(etobicoke_data['Latitude'], etobicoke_data['Longitude'], etobicoke_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#418c3f',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Neigh)  
    
    
folium.features.CircleMarker(
    [Ylatitude, Ylongitude],
    radius=10,
    color='red',
    popup='York',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_Neigh) 

for lat, lng, label in zip(Y_data['Latitude'], Y_data['Longitude'], Y_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3f2199',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Neigh)  
    
folium.features.CircleMarker(
    [ETlatitude, ETlongitude],
    radius=10,
    color='red',
    popup='Scarborough',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_Neigh)     
    
    
for lat, lng, label in zip(ET_data['Latitude'], ET_data['Longitude'], ET_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#ecef32',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Neigh)     
    
map_Neigh

AttributeError: module 'folium.features' has no attribute 'CircleMarker'

In [29]:
folium.features.CircleMarker(
    [CTlatitude, CTlongitude],
    radius=10,
    color='red',
    popup='Central Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_Neigh)

# add markers to map
for lat, lng, label in zip(CT_data['Latitude'], CT_data['Longitude'], CT_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#ea2ada',
        fill=True,
        fill_color='#ea2ada',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Neigh)  
    
    
folium.features.CircleMarker(
    [DTlatitude, DTlongitude],
    radius=10,
    color='red',
    popup='Downtown Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_Neigh) 

for lat, lng, label in zip(DT_data['Latitude'], DT_data['Longitude'], DT_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#0cf9f9',
        fill=True,
        fill_color='#0cf9f9',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Neigh)  
    
    
map_Neigh

AttributeError: module 'folium.features' has no attribute 'CircleMarker'

In [30]:

NY_data = toronto_data[toronto_data['Borough'] == 'North York'].reset_index(drop=True)
NY_data

SC_data = toronto_data[toronto_data['Borough'] == 'Scarborough'].reset_index(drop=True)
SC_data


,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.809196,-79.221701
1,Scarborough,"Guildwood, Morningside, West Hill",43.753480,-79.179995
2,Scarborough,Woburn,43.759824,-79.225291
3,Scarborough,Cedarbrae,43.756467,-79.226692
4,Scarborough,Scarborough Village,43.743742,-79.211632
5,Scarborough,Agincourt,43.785353,-79.278549
6,Scarborough,Upper Rouge,43.809343,-79.187571


<h1>Let's include 2 more Boroughs - Scarborough and North York just to cover larger part of Toronto.</h1>

In [31]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
Slatitude = location.latitude
Slongitude = location.longitude
print('The geograpical coordinate of Scarborough, Toronto are {}, {}.'.format(Slatitude, Slongitude))

The geograpical coordinate of Scarborough, Toronto are 43.773077, -79.257774.


In [32]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
NYlatitude = location.latitude
NYlongitude = location.longitude
print('The geograpical coordinate of North York, Toronto are {}, {}.'.format(NYlatitude, NYlongitude))

The geograpical coordinate of North York, Toronto are 43.7543263, -79.44911696639593.


In [33]:
folium.features.CircleMarker(
    [NYlatitude, NYlongitude],
    radius=10,
    color='red',
    popup='North York',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_Neigh) 

for lat, lng, label in zip(NY_data['Latitude'], NY_data['Longitude'], NY_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#1a4a99',
        fill=True,
        fill_color='#1a4a99',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Neigh)  
    
folium.features.CircleMarker(
    [Slatitude, Slongitude],
    radius=10,
    color='red',
    popup='Scarborough',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_Neigh)     
    
    
for lat, lng, label in zip(SC_data['Latitude'], SC_data['Longitude'], SC_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#68ffd4',
        fill=True,
        fill_color='#68ffd4',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Neigh)     
    
map_Neigh


AttributeError: module 'folium.features' has no attribute 'CircleMarker'

In [34]:
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='#000000',
        popup=label,
        fill = True,
        fill_color='#000000',
        fill_opacity=0.6
    ).add_to(map_Neigh)

map_Neigh

AttributeError: module 'folium.features' has no attribute 'CircleMarker'

<h1>
We can see that Neighbourhoods of Central Toronto and Downtown Toronto has most number of Restaurants.
Let's proceed with the clustering of Neighbourhoods to find the ones similar to the ones in Central Toronto and Downtown Toronto.</h1>

In [35]:
df = etobicoke_data.append(NY_data)
df = df.append(CT_data)
df = df.append(DT_data)
df = df.append(ET_data)
df = df.append(SC_data)

In [36]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
# type your answer here

neigh_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Islington Avenue, Humber Valley Village
Westmount
Alderwood, Long Branch
Northwest, West Humber - Clairville
Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bayview Village
Downsview
Downsview
Humber Summit
Downsview
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
Davisville
Central Bay Street
Rosedale
Church and Wellesley
The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Malvern, Rouge
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Agincourt
Upper Rouge


In [38]:
print(neigh_venues.shape)
neigh_venues.head()

(689, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Islington Avenue, Humber Valley Village",43.664232,-79.531963,Java Joe's Village Cafe,43.662461,-79.532054,Café
1,"Islington Avenue, Humber Valley Village",43.664232,-79.531963,TD Canada Trust,43.662545,-79.531749,Bank
2,"Islington Avenue, Humber Valley Village",43.664232,-79.531963,Shoppers Drug Mart,43.663067,-79.531753,Pharmacy
3,"Islington Avenue, Humber Valley Village",43.664232,-79.531963,Thorncrest Plaza,43.662620,-79.532146,Shopping Mall
4,"Islington Avenue, Humber Valley Village",43.664232,-79.531963,Thorncrest Drug Store,43.662988,-79.531817,Pharmacy


In [39]:
neigh_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,14,14,14,14,14,14
"Alderwood, Long Branch",7,7,7,7,7,7
Bayview Village,12,12,12,12,12,12
Cedarbrae,26,26,26,26,26,26
Central Bay Street,30,30,30,30,30,30
Church and Wellesley,30,30,30,30,30,30
Davisville,30,30,30,30,30,30
Davisville North,30,30,30,30,30,30
Don Mills,60,60,60,60,60,60


In [40]:

print('There are {} uniques categories.'.format(len(neigh_venues['Venue Category'].unique())))

There are 140 uniques categories.


In [41]:
# one hot encoding
neigh_onehot = pd.get_dummies(neigh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neigh_onehot['Neighborhood'] = neigh_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neigh_onehot.columns[-1]] + list(neigh_onehot.columns[:-1])
neigh_onehot = neigh_onehot[fixed_columns]

neigh_onehot.head()

,Neighborhood,American Restaurant,Animal Shelter,Argentinian Restaurant,Asian Restaurant,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Café,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dumpling Restaurant,Electronics Store,Escape Room,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Historic Site,Hobby Shop,Home Service,Hong Kong Restaurant,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Laundry Service,Lingerie Store,Liquor Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Nail Salon,Nightclub,Optical Shop,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pizza Place,Playground,Portuguese Restaurant,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Skating Rink,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Islington Avenue, Humber Valley Village",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Islington Avenue, Humber Valley Village",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Islington Avenue, Humber Valley Village",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Islington Avenue, Humber Valley Village",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Islington Avenue, Humber Valley Village",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [42]:
(neigh_onehot.shape)

(689, 141)

In [43]:
neigh_grouped = neigh_onehot.groupby('Neighborhood').mean().reset_index()
neigh_grouped

,Neighborhood,American Restaurant,Animal Shelter,Argentinian Restaurant,Asian Restaurant,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Café,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dumpling Restaurant,Electronics Store,Escape Room,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Historic Site,Hobby Shop,Home Service,Hong Kong Restaurant,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Laundry Service,Lingerie Store,Liquor Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Nail Salon,Nightclub,Optical Shop,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pizza Place,Playground,Portuguese Restaurant,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Skating Rink,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Agincourt,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.142857,0.000000,0.285714,0.000000,0.000000,0.071429,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000
1,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [44]:
neigh_grouped.shape

(35, 141)

<h3>
Let's get top 5 venues from each Neighbourhood</h3>

In [45]:
num_top_venues = 5

for hood in neigh_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = neigh_grouped[neigh_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                   venue  freq
0     Chinese Restaurant  0.29
1   Cantonese Restaurant  0.14
2   Hong Kong Restaurant  0.07
3       Asian Restaurant  0.07
4  Vietnamese Restaurant  0.07


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1             Gym  0.14
2        Pharmacy  0.14
3     Coffee Shop  0.14
4  Sandwich Place  0.14


----Bayview Village----
                venue  freq
0                Bank  0.17
1        Burger Joint  0.08
2      Breakfast Spot  0.08
3  Persian Restaurant  0.08
4      Sandwich Place  0.08


----Cedarbrae----
                  venue  freq
0  Fast Food Restaurant  0.12
1           Coffee Shop  0.08
2      Department Store  0.04
3                   Gym  0.04
4        Discount Store  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.23
1  Italian Restaurant  0.07
2         Yoga Studio  0.03
3         Pizza Place  0.03
4      Clothing Store  0.03


----Church and Wellesley--

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neigh_grouped['Neighborhood']

for ind in np.arange(neigh_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neigh_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Cantonese Restaurant,Vietnamese Restaurant,Hong Kong Restaurant,Asian Restaurant,Train Station,Food Court,Coffee Shop,Korean Restaurant,Rental Car Location
1,"Alderwood, Long Branch",Pizza Place,Pub,Sandwich Place,Coffee Shop,Pharmacy,Gym,Dance Studio,Deli / Bodega,Department Store,Falafel Restaurant
2,Bayview Village,Bank,Pizza Place,Outdoor Supply Store,Sandwich Place,Burger Joint,Sporting Goods Shop,Breakfast Spot,Fish Market,Fast Food Restaurant,Gas Station
3,Cedarbrae,Fast Food Restaurant,Coffee Shop,Grocery Store,Gym,Bus Station,Sandwich Place,Shopping Mall,Optical Shop,Discount Store,Clothing Store
4,Central Bay Street,Coffee Shop,Italian Restaurant,Department Store,Sandwich Place,Portuguese Restaurant,Clothing Store,Pizza Place,Creperie,Park,Diner


In [48]:
# set number of clusters
kclusters = 21

neigh_grouped_clustering = neigh_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neigh_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([10, 17, 13, 14,  6,  6, 16, 16,  6, 11], dtype=int32)

In [49]:
# add clustering labels
labels = kmeans.labels_.astype(int)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels3', labels)

neigh_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
neigh_merged = neigh_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

neigh_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels3,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Etobicoke,"Islington Avenue, Humber Valley Village",43.664232,-79.531963,12,Pharmacy,Baseball Field,Shopping Mall,Grocery Store,Café,Bank,Yoga Studio,Falafel Restaurant,Escape Room,Electronics Store
1,Etobicoke,Westmount,43.693640,-79.521043,3,Pizza Place,Gas Station,Park,Farmers Market,Falafel Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Dive Bar,Yoga Studio
2,Etobicoke,"Alderwood, Long Branch",43.601717,-79.545232,17,Pizza Place,Pub,Sandwich Place,Coffee Shop,Pharmacy,Gym,Dance Studio,Deli / Bodega,Department Store,Falafel Restaurant
3,Etobicoke,"Northwest, West Humber - Clairville",43.723370,-79.597457,14,Department Store,Chinese Restaurant,Theme Park,Sporting Goods Shop,Bus Station,Food & Drink Shop,Sandwich Place,Mobile Phone Shop,Bank,Fast Food Restaurant
0,North York,Parkwoods,43.761124,-79.324059,20,Pizza Place,Discount Store,Shopping Mall,Fast Food Restaurant,Supermarket,Laundry Service,Café,Chinese Restaurant,Caribbean Restaurant,Coffee Shop


In [50]:
neigh_merged = neigh_merged[~neigh_merged['Cluster Labels3'].isnull()]


# final conversion now worked
neigh_merged[['Cluster Labels3']] = neigh_merged[['Cluster Labels3']].astype(int)

In [51]:

# create map
map_clusters = folium.Map(location=[Tlatitude, Tlongitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neigh_merged['Latitude'], neigh_merged['Longitude'], neigh_merged['Neighbourhood'], neigh_merged['Cluster Labels3']):
    if np.isnan(cluster):
        cluster = np.nan_to_num(cluster)
    
    
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
    

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='#000000',
        popup=label,
        fill = True,
        fill_color='#000000',
        fill_opacity=0.6
    ).add_to(map_clusters)

    
       
map_clusters

AttributeError: module 'folium.features' has no attribute 'CircleMarker'

In [52]:
neigh_merged.loc[neigh_merged['Cluster Labels1'] == 0, neigh_merged.columns[[1] + list(range(5, neigh_merged.shape[1]))]]

KeyError: 'Cluster Labels1'


<h3>Results</h3>
We can see that most of the restaurants are based on Central Toronto and Dowtown Toronto.
The Black circles represents Restaurant locations and the other circles represents the similar clustered Neighbourhoods.
<h3>Conclusion</h3>
Based on above results, we can recommend the following top 3 Neighbourhoods:
1.Adelaide in East Toronto
2.King and Spadina in Downtown Toronto
3.South Hill in Central Toronto